# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![avatar](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=&cache=v2)

### Задания

1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [7]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
query = '''
            SELECT *
            FROM books
'''

In [8]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


## Задача 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [42]:
query = '''
            SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


## Задача 2
Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [39]:
query = '''
            WITH rc AS (SELECT book_id, COUNT(*) AS reviews_count
                        FROM reviews
                        GROUP BY book_id),
            avg_ratings AS (SELECT book_id, AVG(rating) AS avg_rating
                            FROM ratings
                            GROUP BY book_id)
            
            SELECT b.*, rc.reviews_count, ar.avg_rating
            FROM books AS b
            LEFT JOIN avg_ratings AS ar ON b.book_id = ar.book_id
            LEFT JOIN rc ON  b.book_id = rc.book_id
'''

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id,reviews_count,avg_rating
0,652,3,The Body in the Library (Miss Marple #3),191,2006-09-01,50,2.0,4.500000
1,273,352,Galápagos,324,1999-01-12,88,2.0,4.500000
2,51,54,A Tree Grows in Brooklyn,496,2006-05-30,135,5.0,4.250000
3,951,540,Undaunted Courage: The Pioneering First Missio...,592,2003-10-06,269,2.0,4.000000
4,839,332,The Prophet,127,2010-01-01,251,4.0,4.285714
...,...,...,...,...,...,...,...,...
995,672,144,The Cat in the Hat and Other Dr. Seuss Favorites,61,2003-10-14,174,NaN,5.000000
996,83,174,Anne Rice's The Vampire Lestat: A Graphic Novel,404,1991-11-30,33,NaN,3.666667
997,221,151,Essential Tales and Poems,688,2004-10-25,42,NaN,4.000000
998,387,365,Leonardo's Notebooks,352,2005-08-01,49,NaN,4.000000


## Задача 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [64]:
query = '''
            SELECT *
            FROM publishers
            WHERE publisher_id = (SELECT publisher_id
                                FROM books 
                                WHERE num_pages > 50
                                GROUP BY publisher_id
                                ORDER BY COUNT(*) DESC
                                LIMIT 1)
'''

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,212,Penguin Books


## Задача 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [79]:
query = '''
            SELECT author
            FROM books AS b
            JOIN authors AS a ON b.author_id = a.author_id
            WHERE b.book_id = (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(*) >= 50
                                ORDER BY AVG(rating) DESC
                                LIMIT 1)
'''

pd.io.sql.read_sql(query, con = engine) 

,author
0,J.K. Rowling/Mary GrandPré


## Задача 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [117]:
query = '''
            WITH users_50 AS (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(*) >= 50),
                                
            rev_count AS (SELECT username, COUNT(*) AS rev_count
                            FROM reviews
                            GROUP BY username)
                            
            SELECT AVG(rev_count)
            FROM users_50 AS u
            JOIN rev_count AS r ON u.username = r.username
'''

pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.222222
